In [92]:
# Model design
import agentpy as ap

# Visualization and processing
import matplotlib.pyplot as plt
import IPython
import math
import random
import json
import requests

In [93]:
# Define movements keys
UP = 0
DOWN = 1
RIGHT = 2
LEFT = 3

# Define movements values
MOVEMENT_UP = (-1, 0)
MOVEMENT_DOWN = (1, 0)
MOVEMENT_LEFT = (0, -1)
MOVEMENT_RIGHT = (0, 1)

# Colors
    # Ligts
COLOR_RED = 0
COLOR_YELLOW = 1
COLOR_GREEN = 2
    # Car
CAR_COLOR = 3

# Conection Server
URL = "http://localhost:8080/"

#URL = "https://multiagentes-equipo7-server.us-south.cf.appdomain.cloud/"

random.seed(4)

In [94]:
class CarAgent(ap.Agent):
    # Car atributes
    moving = True
    color = CAR_COLOR
    
    def setup(self):
        self.id = 0
        self.x = 0
        self.y = 0
        self.z = 0

        self.crossed = 0
        self.spaces = 0 
        
        self.total_steps_y = 0
        self.total_steps_x = 0
        
        self.direction = random.randint(0,3)
    def get_movement(self):
        if (self.direction == 0):  # Each car has a type of movement
            return MOVEMENT_UP
        elif (self.direction == 1):
          return MOVEMENT_DOWN
        elif (self.direction == 2): 
            return MOVEMENT_RIGHT
        elif (self.direction == 3):
            return MOVEMENT_LEFT

In [95]:
class LightAgent(ap.Agent):
    color  = COLOR_YELLOW
    direction = 0
    def setup(self):
        self.id = 0
    def changeColor(self, light_color):
        self.color = light_color

In [96]:
class Street(ap.Model):
    cars_array = []
    traffic_lights_array = []
    JSON_Object = {}

    global steps
    global red_light_state
    steps = 0
    red_light_state = 0

    def setup(self):
        ################## Agents ##################
        n_traffic_lights = self.p['number_traffic_lights']
        n_cars = self.p['number_cars']
        self.traffic_lights = ap.AgentList(self, n_traffic_lights, LightAgent)
        self.cars = ap.AgentList(self, n_cars, CarAgent)
        ################ Enviroment ################
        x_axes = self.p['size']['X']
        y_axes = self.p['size']['Y']
        self.street = ap.Grid(self, (y_axes,x_axes), torus = False, track_empty= True)

        # Positions
        traffic_lights_positions = self.p['lights_positions']
        road_position = self.p['road_position']
        self.street.add_agents(self.traffic_lights, traffic_lights_positions, empty= False)
        self.street.add_agents(self.cars, random = True, empty = True)

        # Initialize the JSON Objects
        iterator = 0
        for car in self.cars:
            car.id = iterator
            carObject = {
                'id': iterator,
                'x': car.x,
                'y': car.y,
                'z': car.z,
                'direction': car.direction
            }
            self.cars_array.append(carObject)
            iterator += 1 

        iterator = 0
        for traffic_light in self.traffic_lights:
            traffic_light.id = iterator
            lightObject = {
                'id': iterator,
                'state': traffic_light.color
            }
            self.traffic_lights_array.append(lightObject)
            iterator += 1
            
        # Asign a valid position to cars
        for car in self.cars:
            if car.direction == 0:
                x = road_position[0]
                y = random.randint(y_axes-7,y_axes)
                car.x = x
                car.y = y
                self.street.move_to(car, (y, x))
            elif car.direction == 1:
                x = road_position[1]
                y = random.randint(0,7)
                car.x = x
                car.y = y
                self.street.move_to(car, (y, x))
            elif car.direction == 2:
                x = random.randint(0,7)
                y = road_position[2]
                car.x = x
                car.y = y
                self.street.move_to(car, (y, x))
            elif car.direction == 3:
                x = random.randint(x_axes-7,x_axes)
                y = road_position[3]
                car.x = x
                car.y = y
                self.street.move_to(car, (y, x)) 

        self.total_steps_y = self.p['size']['Y'] - car.y
        self.total_steps_x = self.p['size']['X']- car.x

        # Asign direction to each traffic light
        self.traffic_lights[0].direction = UP
        self.traffic_lights[1].direction = DOWN
        self.traffic_lights[2].direction = RIGHT
        self.traffic_lights[3].direction = LEFT
        
    def update_lights_JSON(self, agent_list, new_color):
        _IDs = [] 
        for light in agent_list:
            _IDs.append(light.id)
            #print(light.color)
        for id in _IDs:
            self.traffic_lights_array[id]['state'] = new_color

    def step(self):
        global steps
        global red_light_state

        minTrafficLightDirection = 0
        min = 99999

        # Calculates the minimum distance of the traffic light and the nearest car
        for traffic_light in self.traffic_lights:
            for neighbor in self.street.neighbors(traffic_light, distance= 2):
                position_1 = self.street.positions[neighbor] # tuple(y,x)
                position_2 = self.street.positions[traffic_light]

                # Pythagoras Formula
                # C =sqrt( delta(y1 - y0) + delta(x1, x0)
                distance = math.sqrt( (position_1[0]-position_2[0])**2 + (position_1[1]-position_2[1])**2 )

                if( min > distance and neighbor.direction == traffic_light.direction ):
                    min = distance
                    minTrafficLightDirection = traffic_light.direction
        
        # If no cars are found around a radious of 2 the traffic light will change to yellow
        if min == 99999:
            self.traffic_lights.color = COLOR_YELLOW
            self.update_lights_JSON(self.traffic_lights, COLOR_YELLOW)
        # Turn Green both traffic lights with the closest car on the same direction
        else:
            traffic_light = self.traffic_lights.select(self.traffic_lights.direction == minTrafficLightDirection)
            self.traffic_lights.color = COLOR_RED
            red_light_state += 1
            self.update_lights_JSON(self.traffic_lights, COLOR_RED)
            
            if traffic_light[0].direction == UP or traffic_light[0].direction == DOWN:
                
                green_selection_1 = self.traffic_lights.select(self.traffic_lights.direction == UP)
                green_selection_1.changeColor(COLOR_GREEN)
                self.update_lights_JSON(green_selection_1, COLOR_GREEN)
                green_selection_2 = self.traffic_lights.select(self.traffic_lights.direction == DOWN)
                green_selection_2.changeColor(COLOR_GREEN)
                self.update_lights_JSON(green_selection_2, COLOR_GREEN)
            else:
                green_selection_1 = self.traffic_lights.select(self.traffic_lights.direction == RIGHT)
                green_selection_1.changeColor(COLOR_GREEN)
                self.update_lights_JSON(green_selection_1, COLOR_GREEN)
                green_selection_2 = self.traffic_lights.select(self.traffic_lights.direction == LEFT)
                green_selection_2.changeColor(COLOR_GREEN)
                self.update_lights_JSON(green_selection_2, COLOR_GREEN)
            
        
        iterator = 0
        for car in self.cars.select(self.cars.moving == True):
            for neighbor in self.street.neighbors(car, 2):
                if neighbor.color == COLOR_RED and neighbor.direction == car.direction:
                    break
            else:
                for neighbor in self.street.neighbors(car, 1):
                    if car.direction == UP and neighbor.color == CAR_COLOR and (self.street.positions[neighbor][0]-self.street.positions[car][0]) == -1 and (self.street.positions[neighbor][1]-self.street.positions[car][1]) == 0:
                        if car.crossed == 1 and car.spaces == car.total_steps_y:
                            del self.street.positions[car]
                        break
                    elif car.direction == DOWN and neighbor.color == CAR_COLOR and (self.street.positions[neighbor][0]-self.street.positions[car][0]) == 1 and (self.street.positions[neighbor][1]-self.street.positions[car][1]) == 0:
                        if car.crossed == 1 and car.spaces == car.total_steps_y:
                            del self.street.positions[car]
                        break
                    elif car.direction == RIGHT and neighbor.color == CAR_COLOR and (self.street.positions[neighbor][1]-self.street.positions[car][1]) == 1 and (self.street.positions[neighbor][0]-self.street.positions[car][0]) == 0:
                        if car.crossed == 1 and car.spaces == car.total_steps_x:
                            del self.street.positions[car]
                        break
                    elif car.direction == LEFT and neighbor.color == CAR_COLOR and (self.street.positions[neighbor][1]-self.street.positions[car][1]) == -1 and (self.street.positions[neighbor][0]-self.street.positions[car][0]) == 0:
                        if car.crossed == 1 and car.spaces == car.total_steps_x:
                            del self.street.positions[car]
                        break
                else:
                    if car.spaces == ((self.p['size']['X']/4)*3):
                        car.crossed = 1
                    current_movement = car.get_movement()
                    if(current_movement==MOVEMENT_RIGHT):
                        car.x = car.x+1
                    elif(current_movement==MOVEMENT_LEFT):
                        car.x = car.x-1
                    if(current_movement==MOVEMENT_DOWN):
                        car.y = car.y+1
                    elif(current_movement==MOVEMENT_UP):
                        car.y = car.y-1
                    self.cars_array[iterator]['x'] = car.x
                    self.cars_array[iterator]['y'] = car.y
                    self.street.move_by(car, current_movement)
            iterator += 1

        steps += 1
        if (self.p.steps - 1) == steps:
            print("Tiempo en que los carros llegan a salvo a final de la calle: " + str(self.p.steps) + " segundos.")
            print("Tiempo de espera total en el cruce: " + str(red_light_state) + " segundos.")
            for car in self.cars:
                self.street.remove_agents(car)
                
        self.JSON_Object["cars"] = self.cars_array
        self.JSON_Object["traffic_lights"] = self.traffic_lights_array

        JSON_String = json.dumps(self.JSON_Object)
        print(JSON_String)
        try:
            " "
            requests.post(URL, JSON_String)
        except:
            print("Unable to send positions")            



In [97]:
parameters = {
    'number_traffic_lights': 4,
    'number_cars': 5,
    'lights_positions': [
        (18,20),
        (22,20),
        (20,18),
        (20,22)
    ],
    'road_position': [
        21, # Vertical Right
        19,  # Vertical Left
        21, # Horizontal Bottom
        19   # Horizontal Top
    ],
    'size':{
        'X':40,
        'Y':40
    },
    'steps': 50
}

In [98]:
# Create single-run animation
def animation_plot(model, ax):
  attr_grid = model.street.attr_grid('color')
  color_dict = {0:'#FF0000', 1:'#FFFF00', 2:'#00FF00', 3:'#000000', None:'#5b5b5b'}
  ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
  ax.set_title(f"Simulation of a Street Intersection\n"
                 f"Time-step: {model.t}, Number of cars: "
                 f"{len(model.cars)}")
fig, ax = plt.subplots()
model = Street(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

{"cars": [{"id": 0, "x": 32, "y": 19, "z": 0, "direction": 3}, {"id": 1, "x": 38, "y": 19, "z": 0, "direction": 3}, {"id": 2, "x": 19, "y": 5, "z": 0, "direction": 1}, {"id": 3, "x": 21, "y": 32, "z": 0, "direction": 0}, {"id": 4, "x": 21, "y": 35, "z": 0, "direction": 0}], "traffic_lights": [{"id": 0, "state": 1}, {"id": 1, "state": 1}, {"id": 2, "state": 1}, {"id": 3, "state": 1}]}
{"cars": [{"id": 0, "x": 31, "y": 19, "z": 0, "direction": 3}, {"id": 1, "x": 37, "y": 19, "z": 0, "direction": 3}, {"id": 2, "x": 19, "y": 6, "z": 0, "direction": 1}, {"id": 3, "x": 21, "y": 31, "z": 0, "direction": 0}, {"id": 4, "x": 21, "y": 34, "z": 0, "direction": 0}], "traffic_lights": [{"id": 0, "state": 1}, {"id": 1, "state": 1}, {"id": 2, "state": 1}, {"id": 3, "state": 1}]}
{"cars": [{"id": 0, "x": 30, "y": 19, "z": 0, "direction": 3}, {"id": 1, "x": 36, "y": 19, "z": 0, "direction": 3}, {"id": 2, "x": 19, "y": 7, "z": 0, "direction": 1}, {"id": 3, "x": 21, "y": 30, "z": 0, "direction": 0}, {"id"